In [ ]:
import os
import numpy as np
import xarray as xr
from pathlib import Path

ROOT_DIR = Path("/tera04/zhwei/xionghui/bedrock/data").resolve()
# ROOT_DIR = Path("../data").resolve()
RUN_DIR  = ROOT_DIR / "run"
SHP_DIR  = ROOT_DIR / "Shp"
SAWS_DIR = ROOT_DIR / "SAWS/Kosugi/rawdata"

RES = "500m"
region = [-180, 180, -60, 90]

dtb_path   = ROOT_DIR / "DTB/DTB.nc"
ssoil_path = RUN_DIR / f"Ssoil_{RES}.nc"

DTB_VAR = "Band1" 
SAWS_VAR = "Band1"
DTB_UNIT = "cm"

CHUNKS = {"lat": 1200, "lon": 1200}

# Calculate the DTB stratification (0~5~15~30~60~100~150 cm) to align the SAWS stratification (0~5~15~30~60~100~200 cm)
LAYERS_CM = np.array([0, 5, 15, 30, 60, 100, 150], dtype=np.float32)

def Ssoil():
    dtb = xr.open_dataset(dtb_path, chunks=CHUNKS)[DTB_VAR].astype("float32")

    fv_dtb = dtb.attrs.get("_FillValue", None)
    if fv_dtb is not None:
        dtb = dtb.where(dtb != fv_dtb)
    dtb = dtb.fillna(0)

    saws_list = []
    for i in range(len(LAYERS_CM) - 1):
        f = SAWS_DIR / f"saws{i+1}.nc"
        da = xr.open_dataset(f, chunks=CHUNKS)[SAWS_VAR].astype("float32")

        fv = da.attrs.get("_FillValue", None)
        if fv is not None:
            da = da.where(da != fv)
        da = da.fillna(0)

        saws_list.append(da)

    saws = xr.concat(saws_list, dim="layer")
    saws = saws.assign_coords(layer=np.arange(1, len(LAYERS_CM), dtype=np.int16))

    dtb = dtb.assign_coords(lat=saws["lat"], lon=saws["lon"])

    top = xr.DataArray(LAYERS_CM[:-1], dims="layer")
    bot = xr.DataArray(LAYERS_CM[1:],  dims="layer")
    dz  = bot - top

    thk = (dtb - top).clip(min=0)
    thk = xr.where(thk > dz, dz, thk)

    # Ssoil (mm) = sum( SAWS * thk(cm) * 10 )
    # (cm -> mm) multiply by 10
    ssoil_mm = (saws * thk * 10.0).sum("layer").astype("float32")

    out = xr.Dataset({"Ssoil": ssoil_mm})
    out["Ssoil"].attrs["units"] = "mm"
    out["Ssoil"].attrs["long_name"] = "Total plant-available soil water (integrated to bedrock depth, capped at 150 cm)"
    out["Ssoil"].attrs["layers_cm"] = "0-5,5-15,15-30,30-60,60-100,100-150"
    out["Ssoil"].attrs["dtb_unit_used"] = "cm"

    enc = {
    "Ssoil": {
        "zlib": True, "complevel": 3,
        "dtype": "float32",
        "chunksizes": (CHUNKS["lat"], CHUNKS["lon"]),
        "_FillValue": np.float32(-9999),
    }
    }
    out.to_netcdf(ssoil_path, encoding=enc)

if __name__ == "__main__":
    Ssoil()


In [1]:
import os
import numpy as np
import xarray as xr
from pathlib import Path

ROOT_DIR = Path("/tera04/zhwei/xionghui/bedrock/data").resolve()
# ROOT_DIR = Path("../data").resolve()
RUN_DIR  = ROOT_DIR / "run"
SHP_DIR  = ROOT_DIR / "Shp"
SAWS_DIR = ROOT_DIR / "SAWS/Kosugi"

RES = "p05"
region = [-180, 180, -60, 90]

dtb_path   = ROOT_DIR / f"DTB/Pelletier/DTB_Pelletier_{RES}_cm.nc"
ssoil_path = RUN_DIR / f"Ssoil_{RES}.nc"

DTB_VAR = "Band1" 
SAWS_VAR = "Band1"
DTB_UNIT = "cm"

CHUNKS = {"lat": 1200, "lon": 1200}

# Calculate the DTB stratification (0~5~15~30~60~100~150 cm) to align the SAWS stratification (0~5~15~30~60~100~200 cm)
LAYERS_CM = np.array([0, 5, 15, 30, 60, 100, 150], dtype=np.float32)

def Ssoil():
    dtb = xr.open_dataset(dtb_path, chunks=CHUNKS)[DTB_VAR].astype("float32")

    fv_dtb = dtb.attrs.get("_FillValue", None)
    if fv_dtb is not None:
        dtb = dtb.where(dtb != fv_dtb)
    dtb = dtb.fillna(0)

    saws_list = []
    for i in range(len(LAYERS_CM) - 1):
        f = SAWS_DIR / f"SAWS_Kosugi_{RES}_m3m-3_{i+1}.nc"
        da = xr.open_dataset(f, chunks=CHUNKS)[SAWS_VAR].astype("float32")

        fv = da.attrs.get("_FillValue", None)
        if fv is not None:
            da = da.where(da != fv)
        da = da.fillna(0)

        saws_list.append(da)

    saws = xr.concat(saws_list, dim="layer")
    saws = saws.assign_coords(layer=np.arange(1, len(LAYERS_CM), dtype=np.int16))

    dtb = dtb.assign_coords(lat=saws["lat"], lon=saws["lon"])

    top = xr.DataArray(LAYERS_CM[:-1], dims="layer")
    bot = xr.DataArray(LAYERS_CM[1:],  dims="layer")
    dz  = bot - top

    thk = (dtb - top).clip(min=0)
    thk = xr.where(thk > dz, dz, thk)

    # Ssoil (mm) = sum( SAWS * thk(cm) * 10 )
    # (cm -> mm) multiply by 10
    ssoil_mm = (saws * thk * 10.0).sum("layer").astype("float32")

    out = xr.Dataset({"Ssoil": ssoil_mm})
    out["Ssoil"].attrs["units"] = "mm"
    out["Ssoil"].attrs["long_name"] = "Total plant-available soil water (integrated to bedrock depth, capped at 150 cm)"
    out["Ssoil"].attrs["layers_cm"] = "0-5,5-15,15-30,30-60,60-100,100-150"
    out["Ssoil"].attrs["dtb_unit_used"] = "cm"

    enc = {
    "Ssoil": {
        "zlib": True, "complevel": 3,
        "dtype": "float32",
        "chunksizes": (CHUNKS["lat"], CHUNKS["lon"]),
        "_FillValue": np.float32(-9999),
    }
    }
    out.to_netcdf(ssoil_path, encoding=enc)

if __name__ == "__main__":
    Ssoil()
